In [1]:
%cd .. 


g:\AI\Electro_Pi\Deploy_Chest_X_Ray_Session\X-Ray-Classification


In [2]:
import pandas as pd 

data_map = pd.read_csv("data/data_map.csv")
data_map.head()

,imgs_paths,clas,split,resized
0,data\chest_xray\test\NORMAL\IM-0001-0001.jpeg,NORMAL,test,data/resized/test_NORMAL_IM-0001-0001.jpeg
1,data\chest_xray\test\NORMAL\IM-0003-0001.jpeg,NORMAL,test,data/resized/test_NORMAL_IM-0003-0001.jpeg
2,data\chest_xray\test\NORMAL\IM-0005-0001.jpeg,NORMAL,test,data/resized/test_NORMAL_IM-0005-0001.jpeg
3,data\chest_xray\test\NORMAL\IM-0006-0001.jpeg,NORMAL,test,data/resized/test_NORMAL_IM-0006-0001.jpeg
4,data\chest_xray\test\NORMAL\IM-0007-0001.jpeg,NORMAL,test,data/resized/test_NORMAL_IM-0007-0001.jpeg


In [6]:
from sklearn.preprocessing import LabelEncoder 
lbl_encoder=LabelEncoder()
data_map["clas_enc"]= lbl_encoder.fit_transform(data_map["clas"])

In [7]:
import torch
torch.__version__


'2.4.0+cpu'

In [8]:
import torch
from torch import nn 
from torch.utils.data import  Dataset , DataLoader
from torchvision import transforms , models
from torchvision import transforms
from PIL import Image



In [9]:
class XrayDataset(Dataset):
    def __init__(self, df_split, transform):
        self.df_split = df_split
        self.transform = transform

    def __len__(self):
        return len(self.df_split)

    def __getitem__(self, index):
        img_path = self.df_split.iloc[index]["resized"]
        clas = self.df_split.iloc[index]["clas_enc"]

        img = Image.open(img_path).convert("L")
        img_trans = self.transform(img)

        return img_trans, clas


In [10]:
train_dataset = XrayDataset(data_map[data_map["split"]== "train"],transform=transforms.ToTensor())
val_dataset = XrayDataset(data_map[data_map["split"]== "val"],transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size= 32 , shuffle = True)
val_loader = DataLoader(val_dataset, batch_size= 32 , shuffle = True)





In [11]:
model = models.resnet18(weights= models.ResNet18_Weights.IMAGENET1K_V1)
model.conv1= nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc= nn.Linear(in_features=512, out_features=2, bias=True)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
model = model.to(device)

In [15]:
optimizer = torch.optim.Adam(model.parameters())
loss = nn.CrossEntropyLoss()
from tqdm import tqdm

In [17]:
for epoch in range(2):
    model.train()
    running_loss = 0 
    for imgs, lbls in tqdm(train_loader, total=len(train_loader)):
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad()
        preds = model(imgs)
        loss_val = loss(preds, lbls.long())   # ← تحويل الوسوم إلى Long
        loss_val.backward()
        optimizer.step()
        running_loss += loss_val.item()

    print("Epoch Loss:", running_loss / len(train_loader))


100%|██████████| 163/163 [09:18<00:00,  3.43s/it]


Epoch Loss: 0.16043991501517366


100%|██████████| 163/163 [06:41<00:00,  2.46s/it]

Epoch Loss: 0.06977242531334879


In [18]:
import os
os.makedirs("data/models", exist_ok=True)
torch.save(model.state_dict(), "data/models/model.pth")



In [ ]:


def infer(img):
    img_gray = img.convert("L") 
    img_trans = transforms.ToTensor()(img_gray).unsqueeze(0).to(device)  
    preds = model(img_trans) 
    clas_index = torch.argmax(preds).item()
    final_pred = lbl_encoder.inverse_transform([clas_index])[0]  # 5️⃣ تحويل الفهرس إلى اسم الفئة الأصلي باستخدام الـ 
    return final_pred 

img_path = r"data\resized\train_NORMAL_NORMAL2-IM-1188-0001.jpeg"
img = Image.open(img_path)
infer(img)



'PNEUMONIA'

In [20]:
import pickle 

with open("data\lbl_encoder.pkl", "wb") as f : 
    pickle.dump(LabelEncoder, f)